In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import numpy as np
import pandas as pd
from avgn.utils.paths import DATA_DIR, most_recent_subdirectory, ensure_dir
from sklearn.preprocessing import scale
#from pyclustertend import hopkins, vat, ivat
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

In [14]:
from avgn.clusterability.hopkins import hopkins_statistic

In [15]:
datasets = list(DATA_DIR.glob('embeddings/*/*.pickle'))
len(datasets)

98

In [16]:
datasets

[PosixPath('/mnt/cube/tsainbur/Projects/github_repos/avgn_paper/data/embeddings/koumura_bengalese_finch/Bird0.pickle'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/avgn_paper/data/embeddings/koumura_bengalese_finch/Bird2.pickle'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/avgn_paper/data/embeddings/koumura_bengalese_finch/Bird9.pickle'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/avgn_paper/data/embeddings/koumura_bengalese_finch/Bird4.pickle'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/avgn_paper/data/embeddings/koumura_bengalese_finch/Bird6.pickle'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/avgn_paper/data/embeddings/koumura_bengalese_finch/Bird10.pickle'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/avgn_paper/data/embeddings/koumura_bengalese_finch/Bird5.pickle'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/avgn_paper/data/embeddings/koumura_bengalese_finch/Bird8.pickle'),
 PosixPath('/mnt/cube/tsainbur/Projects

In [17]:
def remove_outliers(z, pct = 99.5):
    """ GPU based UMAP algorithm produces some outliers that UMAP does not, but is much faster
    this is a quick fix for that. 
    """
    _min = np.percentile(z, (100-pct), axis=0)
    _max = np.percentile(z, pct, axis=0)
    for col in range(np.shape(z)[1]):
        mask = z[:,col] < _min[col]
        z[mask,col] = _min[col]
        mask = z[:,col] > _max[col]
        z[mask,col] = _max[col]
    return z

In [22]:
??hopkins_statistic

In [18]:
def gen_clusterability_df(dataset):
    """
    m_prop_n is the number of samples over X to perform hopkins statistic on (.1 is reccomended)
    """
    save_loc = DATA_DIR / 'clusterability' / 'convex_sample' / \
        (dataset.parent.stem + '_ ' + dataset.stem + '.pickle')

    if save_loc.exists():
        return

    ds = pd.read_pickle(dataset)
    specs = np.stack(ds[['spectrogram']].spectrogram.values)
    specs = specs.reshape(len(specs), -1)
    specs = scale(specs)

    umap_proj = np.vstack(ds[['umap']].umap.values)
    umap_proj = remove_outliers(umap_proj, pct=99.5)
    umap_proj = scale(umap_proj)

    print((dataset, np.shape(umap_proj)))
    fig, ax = plt.subplots()
    ax.scatter(umap_proj[:, 0], umap_proj[:, 1], s=1, color='k', alpha=0.1)
    plt.show()

    hopkins_dict = {
        'umap':
        {
            0.01: hopkins_statistic(umap_proj, m_prop_n=0.01, n_neighbors=1, distribution="uniform_convex_hull"),
            0.1: hopkins_statistic(umap_proj, m_prop_n=0.1, n_neighbors=1, distribution="uniform_convex_hull"),
        },

    }

    clusterability_df = pd.DataFrame([[dataset, dataset.parent.stem, dataset.stem,
                                       hopkins_dict['umap'][0.01], hopkins_dict['umap'][0.1], ]], columns=[
        'df', 'dataset', 'indv',
        'umap_hopkins_1', 'umap_hopkins_10'
    ])

    clusterability_df.to_pickle(save_loc)

In [19]:
ensure_dir(DATA_DIR / 'clusterability' / 'convex_sample')

In [20]:
from joblib import Parallel, delayed

In [21]:
clust_data = Parallel(n_jobs=-1, verbose=10)(
    delayed(gen_clusterability_df)(dataset)
    for dataset in tqdm(datasets)
)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   11.5s


[Parallel(n_jobs=-1)]: Done  45 out of  98 | elapsed:   12.3s remaining:   14.5s
[Parallel(n_jobs=-1)]: Done  55 out of  98 | elapsed:   14.5s remaining:   11.4s
[Parallel(n_jobs=-1)]: Done  65 out of  98 | elapsed:   15.7s remaining:    8.0s
[Parallel(n_jobs=-1)]: Done  75 out of  98 | elapsed:   22.1s remaining:    6.8s
[Parallel(n_jobs=-1)]: Done  85 out of  98 | elapsed:   26.5s remaining:    4.1s
[Parallel(n_jobs=-1)]: Done  95 out of  98 | elapsed:   34.1s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  98 out of  98 | elapsed:  2.4min finished
